### https://github.com/tensorflow/ranking

블로그 : https://ai.googleblog.com/2018/12/tf-ranking-scalable-tensorflow-library.html

In [1]:
# code download
!git clone https://github.com/tensorflow/ranking.git

Cloning into 'ranking'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 68 (delta 22), reused 54 (delta 12), pack-reused 0
Unpacking objects: 100% (68/68), done.


## bazel install for building tf-ranking package

https://stackoverflow.com/questions/45767275/unable-to-install-bazel-on-ubuntu-14-04-using-apt-get 참조


In [5]:
# install bazel
!apt-get install pkg-config zip g++ zlib1g-dev unzip python
!wget https://github.com/bazelbuild/bazel/releases/download/0.20.0/bazel-0.20.0-installer-linux-x86_64.sh
!chmod +x bazel-0.20.0-installer-linux-x86_64.sh
!./bazel-0.20.0-installer-linux-x86_64.sh --user

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
python is already the newest version (2.7.15~rc1-1).
unzip is already the newest version (6.0-21ubuntu1).
zip is already the newest version (3.0-11build1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
g++ is already the newest version (4:7.3.0-3ubuntu2.1).
g++ set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [11]:
# building tf-ranking package
import os
os.environ['PATH']
os.environ['PATH'] = os.environ['PATH']+":$HOME/bin"
!export PATH="$PATH:$HOME/bin"
!apt-get install python-pip

!cd ranking  && $HOME/bin/bazel build /tensorflow_ranking/tools/pip_package:build_pip_package
!cd ranking  && tensorflow_ranking/tools/pip_package/build_pip_package.sh /tmp/ranking_pip
!pip install /tmp/ranking_pip/tensorflow_ranking*.whl

'/usr/local/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin'

In [49]:
# test after installing tf-ranking package
!cd ranking && /root/bin/bazel test //tensorflow_ranking/...

INFO: Invocation ID: 55fef6c4-2131-4f2b-b06e-b8fc86186a10
Loading: 
Loading: 0 packages loaded
Analyzing: 21 targets (1 packages loaded, 0 targets configured)
Analyzing: 21 targets (5 packages loaded, 31 targets configured)
INFO: Analysed 21 targets (7 packages loaded, 181 targets configured).
INFO: Found 13 targets and 8 test targets...
[13 / 15] 2 actions running
    Testing //.../python:feature_test; 0s processwrapper-sandbox
[13 / 15] 2 actions running
    Testing //.../python:feature_test; 1s processwrapper-sandbox
[13 / 15] 2 actions running
    Testing //.../python:feature_test; 2s processwrapper-sandbox
[13 / 15] 2 actions running
    Testing //.../python:feature_test; 3s processwrapper-sandbox
[17 / 19] 1 / 8 tests; 2 actions running; last test: .../python:utils_test
    Testing //.../python:feature_test; 3s processwrapper-sandbox
[17 / 19] 1 / 8 tests; 2 actions running; last test: .../python:utils_test
    Testing //.../python:feature_test; 3s processwrapper-sandbox
[17 / 19

In [0]:
!python -c "import tensorflow_ranking"

### Train test

In [0]:
os.environ['OUTPUT_DIR']="/tmp/output"
os.environ['TRAIN']="ranking/tensorflow_ranking/examples/data/train.txt"
os.environ['VALI']="ranking/tensorflow_ranking/examples/data/vali.txt "
os.environ['TEST']="ranking/tensorflow_ranking/examples/data/test.txt"

In [0]:
!rm -rf $OUTPUT_DIR

In [72]:
!/root/bin/bazel build -c opt ranking/tensorflow_ranking/examples/tf_ranking_libsvm_py_binary

ERROR: The 'build' command is only supported from within a workspace.
INFO: Invocation ID: 6de02bb4-dd02-45ad-97c1-e477fa783a4b


In [73]:
!ranking/bazel-bin/tensorflow_ranking/examples/tf_ranking_libsvm_py_binary \
  --train_path=$TRAIN \
  --vali_path=$VALI \
  --test_path=$TEST \
  --output_dir=$OUTPUT_DIR \
  --num_features=136 \
  --num_train_steps=100

INFO:tensorflow:Loading data from ranking/tensorflow_ranking/examples/data/train.txt
INFO:tensorflow:Number of queries: 27
INFO:tensorflow:Number of documents in total: 119
INFO:tensorflow:Number of documents discarded: 0
INFO:tensorflow:Loading data from ranking/tensorflow_ranking/examples/data/vali.txt
INFO:tensorflow:Number of queries: 9
INFO:tensorflow:Number of documents in total: 36
INFO:tensorflow:Number of documents discarded: 0
INFO:tensorflow:Loading data from ranking/tensorflow_ranking/examples/data/test.txt
INFO:tensorflow:Number of queries: 9
INFO:tensorflow:Number of documents in total: 36
INFO:tensorflow:Number of documents discarded: 0
INFO:tensorflow:Using config: {'_model_dir': '/tmp/output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_che

## Notebook example practice

In [80]:
import tensorflow as tf
import tensorflow_ranking as tfr

tf.enable_eager_execution()
tf.executing_eagerly()

True

Next, we will define some global parameters.

In [0]:
# Store the paths to files containing training and test instances.
# As noted above, we will assume the data is in the LibSVM format
# and that the content of each file is sorted by query ID.

# file path : ranking/tensorflow_ranking/examples

_TRAIN_DATA_PATH="ranking/tensorflow_ranking/examples/data/train.txt"
_TEST_DATA_PATH="ranking/tensorflow_ranking/examples/data/test.txt"

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS="pairwise_logistic_loss"

# In the TF-Ranking framework, a training instance is represented
# by a Tensor that contains features from a list of documents
# associated with a single query. For simplicity, we fix the shape
# of these Tensors to a maximum list size and call it "list_size,"
# the maximum number of documents per query in the dataset.
# In this demo, we take the following approach:
#   * If a query has fewer documents, its Tensor will be padded
#     appropriately.
#   * If a query has more documents, we shuffle its list of
#     documents and trim the list down to the prescribed list_size.
_LIST_SIZE=100

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.
_NUM_FEATURES=136

# Parameters to the scoring function.
_BATCH_SIZE=32
_HIDDEN_LAYER_DIMS=["20", "10"]

### Input Pipeline

첫 번째 단계는 데이터 집합을 읽는 입력 파이프 라인을 만들고 tensorflow.data.Dataset 개체를 생성합니다. 이 예제에서는 주어진 파일에서 Dataset을 생성하기 위해 tensorflow_ranking.data 모듈에 포함 된 LibSVM 파서를 호출합니다.

이 함수를 경로 인수로 매개 변수화하여 함수가 학습 및 테스트 데이터 파일을 읽는 데 사용될 수 있도록합니다.

In [0]:
def input_fn(path):
  train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
  )

  train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
  return train_dataset.make_one_shot_iterator().get_next()

### Scoring Function

다음으로, 우리는 틀림없이 TF 랭킹 모델의 중심에있는 Scoring function 으로 돌아갑니다. 아이디어는 (일련의) 쿼리 - 문서 쌍 (들)에 대한 관련성 점수를 계산하는 것입니다. TF- 랭킹 모델은 훈련 데이터를 사용하여이 기능을 학습합니다.
여기에서는 피드 포워드 네트워크를 사용하여 채점 함수를 공식화합니다. 함수는 단일 example (즉, 질의 - 문서 쌍)의 특징을 취하여 관련도를 산출한다.

In [0]:
def example_feature_columns():
  """Returns the example feature columns."""
  feature_names = [
      "%d" % (i+1) for i in range(0, _NUM_FEATURES)
  ]
  return {
      name: tf.feature_column.numeric_column(
        name, shape=(1,), default_value=0.0) for name in feature_names
  }

def make_score_fn():
  """Returns a scoring function to build `EstimatorSpec`."""

  def _score_fn(context_features, group_features, mode, params, config):
    """Defines the network to score a documents."""
    del params
    del config
    # Define input layer.
    example_input = [
        tf.layers.flatten(group_features[name])
        for name in sorted(example_feature_columns())
    ]
    input_layer = tf.concat(example_input, 1)

    cur_layer = input_layer
    for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
      cur_layer = tf.layers.dense(
          cur_layer,
          units=layer_width,
          activation="tanh")

    logits = tf.layers.dense(cur_layer, units=1)
    return logits

  return _score_fn

### Evaluation Metrics

우리는 TF 랭킹 라이브러리에서 인기있는 정보 검색 평가 메트릭을 구현했습니다.

In [0]:
def eval_metric_fns():
  """Returns a dict from name to metric functions.

  This can be customized as follows. Care must be taken when handling padded
  lists.

  def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
  metric_fns["auc"] = _auc

  Returns:
    A dict mapping from metric name to a metric function with above signature.
  """
  metric_fns = {}
  metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
  })

  return metric_fns

Putting It All Together

우리는 이제 위의 모든 구성 요소를 함께 모으고 모델을 훈련하고 평가하는 데 사용할 수있는 Estimator를 만들 준비가되었습니다.

In [0]:
def get_estimator(hparams):
  """Create a ranking estimator.

  Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

  Returns:
    tf.learn `Estimator`.
  """
  def _train_op_fn(loss):
    """Defines train op used in ranking head."""
    return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.train.get_global_step(),
        learning_rate=hparams.learning_rate,
        optimizer="Adagrad")

  ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

  return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
      params=hparams)

위에 정의한 Estimator를 인스턴스화하고 초기화합니다

In [104]:
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpxlbzj587', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3fa22a51d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [131]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpxlbzj587/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpxlbzj587/model.ckpt.
INFO:tensorflow:loss = 0.061991654, step = 101
INFO:tensorflow:global_step/sec: 5.07503
INFO:tensorflow:loss = 0.018773297, step = 201 (19.708 sec)
INFO:tensorflow:global_step/sec: 5.25079
INFO:tensorflow:loss = 0.011238843, step = 301 (19.045 sec)
INFO:tensorflow:global_step/sec: 5.3784
INFO:tensorflow:loss = 0.0074596484, step = 401 (18.591 sec)
INFO:tensorflow:global_step/sec: 5.29672
INFO:tensorflow:loss = 0.0057899323, step = 501 (18.884 sec)
INFO:tensorflow:global_step/sec: 5.35089
INFO:tensorflow:loss = 0.004536319, step = 601 (18.686 sec)
INFO:tensorflow:global_step/sec: 5.328
INFO:tensorflow:loss = 0.0037323814, step = 701 (18.770 sec)
INFO:tensorflow:global_step/sec: 5.36255
INFO:tensorflow:loss = 0.002971959,

In [132]:
ranker.model_dir

'/tmp/tmpxlbzj587'

In [0]:
LOG_DIR = '/tmp/tmpxlbzj587'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
# ! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# ! unzip ngrok-stable-linux-amd64.zip

In [149]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://d5f572f4.ngrok.io
